In [91]:
import os
import tweepy
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [92]:
consumer_key = os.environ["TWITTER_API_KEY"]
consumer_secret = os.environ["TWITTER_API_KEY_SECRET"]
access_token = os.environ["TWITTER_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_ACCESS_TOKEN_SECRET"]

In [93]:
auth = tweepy.OAuth1UserHandler(
    consumer_key, 
    consumer_secret,
    access_token,
    access_token_secret
    )

api = tweepy.API(auth)
api


In [94]:
me = api.verify_credentials()
# me


In [95]:
search_country = 'India'

In [143]:
def scrape_tweets(search_words, date_since, date_until, num_tweets, num_runs, search_country, place_id):
    
    tweets_df = pd.DataFrame(columns = ['user_id', 'tweet_id', 'username', 'location', 
                                        'following', 'followers', 'twt_created_at', 'total_tweets', 
                                        'retweet_count', 'text', 'hashtags', 'mentions'])
    
    program_start = time.time()
    
    
    
    for i in range(0, num_runs):
        
        start_run = time.time()
        tweets = tweepy.Cursor(api.search_tweets, q=search_words + ' AND ' + ("place:%s" % place_id), lang='en', tweet_mode='extended').items(num_tweets)
        
    
        
        tweet_list = [tweet for tweet in tweets]

        no_tweet = 0
        
        for tweet in tweet_list:
            # print(datetime.strptime(str(tweet.created_at), '%Y-%m-%d'))
            # print(tweet.created_at.date())
            # print(tweet.created_at.date() > datetime.strptime(date_since, '%Y-%m-%d').date())
            # print(tweet.created_at.date() < datetime.strptime(date_until, '%Y-%m-%d').date())
            
            
            # print(type(tweet.created_at))
            if tweet.created_at.date() <= datetime.strptime(date_until, '%Y-%m-%d').date():
            # print("In here")
                user_id = tweet.user.id
                tweet_id = tweet.id
                username = tweet.user.screen_name
                # acctdesc = tweet.user.description
                location = tweet.user.location
                following = tweet.user.friends_count
                followers = tweet.user.followers_count
                twt_created_at = tweet.created_at
                total_tweets = tweet.user.statuses_count
                retweet_count = tweet.retweet_count
                hashtags = tweet.entities['hashtags']
                mentions = tweet.entities['user_mentions']
            
                try:    
                    text = tweet.retweeted_status.full_text
                except AttributeError:
                    text = tweet.full_text
                    # print(text)
                ith_tweet = [user_id, tweet_id, username, location, 
                            following, followers, twt_created_at ,total_tweets, 
                            retweet_count, text, hashtags, mentions]
                
                tweets_df.loc[len(tweets_df)] = ith_tweet
        
                no_tweet += 1
        
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('Number of tweets scraped for run {} is {}'.format(i+1, no_tweet))
        print('Time take for {} run to complete is {} mins'.format(i+1, duration_run))
    
        time.sleep(920)
    
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_tweets.csv'
    tweets_df.to_csv(filename, index=False)
    
    program_end = time.time()
    print('Scraping is completed.')    
    print('Total time taken to scrape is {} minutes.'.format(round(program_end-program_start)/60, 2))
    
    return tweets_df
    


In [144]:
search_words = "#governmenthospital OR #hospital OR #patientdeath OR #medicalnegligence OR @MoHFW_INDIA OR #medicalmalpractice OR #medicalerrors OR #lackofmedicalcare"

date_since = '2022-01-01'
date_until = '2022-10-30'
num_tweets = 2500
num_runs = 6
places = api.search_geo(query=search_country, granularity="country")
place_id = places[0].id

scrape_tweets(search_words, date_since, date_until, num_tweets, num_runs, search_country, place_id)

Number of tweets scraped for run 1 is 218
Time take for 1 run to complete is 0.19 mins
Number of tweets scraped for run 2 is 215
Time take for 2 run to complete is 0.18 mins
Number of tweets scraped for run 3 is 215
Time take for 3 run to complete is 0.2 mins
Number of tweets scraped for run 4 is 215
Time take for 4 run to complete is 0.21 mins
Number of tweets scraped for run 5 is 215
Time take for 5 run to complete is 0.18 mins
Number of tweets scraped for run 6 is 215
Time take for 6 run to complete is 0.19 mins
